In [ ]:
!pip install -U langchain langchain_core langchain_community

In [ ]:
!pip install langchain_google_genai
!pip install langchain-text-splitter


ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter


In [ ]:
pip install pinecone

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 10.7 MB/s eta 0:00:00


In [ ]:
import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")




Enter your Google API key: ··········


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain_core.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

In [ ]:
from pinecone import Pinecone, ServerlessSpec


In [ ]:
import getpass
import os

pinecone_api_key = getpass.getpass("Enter your Pinecone API key: ")
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
# gemini setup
import getpass
import os
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")

In [ ]:
index_name = "bge-base-index"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,   # bge-base-en-v1.5 outputs 768-dim vectors
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [ ]:

embedder = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

vector = embedder.embed_query("Pinecone is a vector database")
index = pc.Index(index_name)
index.upsert(vectors=[("1", vector, {"text": "Pinecone is a vector database"})])


/tmp/ipython-input-210017672.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'upserted_count': 1}

In [ ]:
#data ingestion
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader("/content/kebo114.pdf")
docs=loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_spliter.split_documents(docs)


In [ ]:
embedder = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5",
    model_kwargs={"device": "cpu"},  # use "cuda" if GPU available
    encode_kwargs={"normalize_embeddings": True}
)

vectors = []
for i, doc in enumerate(docs):
    vec = embedder.embed_query(doc.page_content)
    vectors.append((str(i), vec, {"text": doc.page_content}))


In [ ]:
index.upsert(vectors=vectors)


{'upserted_count': 12}